In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
import os

In [2]:
#reading the document dataframe which cluster it belongs to
#import the document_clustered.csv file
dataFrame = pd.read_csv('../document_clustered.csv',sep=',',index_col=None)

In [3]:
dataFrame.head()

,category,content,clusters,document
0,0,added oilseed processing mill standstill since...,42,0000045
1,0,march traded volumes uk grain potato futures f...,34,0008858
2,0,trades reported far us soymeal indicated uncha...,44,0001242
3,0,exported january european community countries ...,21,0009002
4,0,foreign meal users waiting expected surge ship...,44,0009026


In [4]:
#lets train the model with KNeighbourclassifier
X= dataFrame['content']
y= dataFrame['clusters'] #type is pandas.core.series.Series

In [5]:
#import necessary libraries from scikit learn
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
import re

In [6]:
def tokenize(text):
    words = [words.lower() for words in nltk.word_tokenize(text)]
    filter_tokens = []
    for tokens in words:
        if re.search('[a-zA-z]',tokens):
            filter_tokens.append(tokens)
    return filter_tokens

In [7]:
#TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=1.0,max_features=10000,
                                   use_idf=True,tokenizer=tokenize,ngram_range=(1,3))
%time tfidf_matrix = tfidf_vectorizer.fit_transform(X)
print(tfidf_matrix.shape)

CPU times: user 22.7 s, sys: 176 ms, total: 22.9 s
Wall time: 23.1 s
(11475, 10000)


In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.3)

In [9]:
print(len(X_train),len(X_test),len(Y_train),len(Y_test)) #total dataset is splitted as 80,70

8032 3443 8032 3443


In [10]:
#Pipeline
knn_clf = Pipeline([('vect',TfidfVectorizer()),('clf',KNeighborsClassifier(n_neighbors=50))])
%time knn_fit = knn_clf.fit(X_train,Y_train).predict(X_test)
accuracy = np.mean(knn_fit == Y_test)

CPU times: user 4.13 s, sys: 540 ms, total: 4.67 s
Wall time: 5.58 s


In [11]:
print("Accuracy using KNeighbors is {}".format(accuracy * 100))

Accuracy using KNeighbors is 75.86407203020622


In [12]:
#Random Forest Classifier
from sklearn .ensemble import RandomForestClassifier
%time rndm_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',RandomForestClassifier(n_estimators=20,criterion='gini',oob_score=True))])

rndm_predict = rndm_clf.fit(X_train,Y_train).predict(X_test)
rndm_score = np.mean(rndm_predict==Y_test)

print("Accuracy using Random Forest Classifier is:{}".format(rndm_score * 100))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 251 µs
Accuracy using Random Forest Classifier is:80.2788266047052


In [14]:
from sklearn import svm
%time svc_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',svm.SVC(C=1.0,kernel='linear'))])
svc_predict = svc_clf.fit(X_train,Y_train).predict(X_test)
svc_score = np.mean(svc_predict==Y_test)

print("Accuracy using Support Vector Machine is :{}".format(svc_score * 100))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 255 µs
Accuracy using Support Vector Machine is :93.40691257624165


In [ ]:
from sklearn.externals import joblib
joblib.save(s)